<a href="https://colab.research.google.com/github/shahabbadihi/IT/blob/master/content-based-evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sklearn
import pandas as pd
import requests
import io

file_url = "https://raw.githubusercontent.com/shahabbadihi/IT/master/PreprocessedDataset.csv"
response = requests.get(file_url)

dest = 'local-file.csv'

with open(dest, "wb") as filee:
  filee.write(response.content)

df = pd.read_csv(dest)
df=df.fillna('')
df.head(74998)


,product_id,product_title,title_en,user_id,likes,dislikes,verification_status,recommend,title,comment,advantages,disadvantages,cart_id
0,559850,خاک و کود تی تی بسته 10 کیلوگرمی,TC,466132,2,0,verified,not_recommended,خاک عاری از مواد مقوی,اصلا خاک خوبی نبود. کلی سنگ های درشت داشت و اص...,"[""ندارد""]","[""پر از سنگ ریزه ""]",1
1,112946,توستر فلر مدل TO 220,HA,466139,2,0,verified,not_recommended,خراب شدن اهرم پایین برنده,بعد از دو و نیم سال استفاده، اهرم پایین دهنده ...,"[""زیبایی، سبکی و اندازه مناسب توستر""]","[""خراب شدن اهرم پایین برنده""]",2
2,607671,هدست واقعیت مجازی ایکس پی پروداکت مدل VR BOX,AC,466160,1,2,verified,no_idea,قیمت پایین فدای کیفیت .....,کیفیت لنزش پایینه شرکت فروشنده گارانتی تعویض ن...,"[""ارزون""]","[""کیفیت پایین لنز""]",3
3,757892,کیف پول زنانه آیسا مدل KZP-1,FA,466160,1,0,verified,not_recommended,کیفیت ساخت پایین,اصلا با قیمت فروش ویژه هم ارزش خرید نداره_x000...,,"[""کیفیت ساخت پایین\r"",""چسب کاری داخل و بیرون""]",3
4,601446,اسپیکر قابل حمل مدل ip085 مناسب برای آیفون و ...,IT,466160,2,2,verified,no_idea,بدرد ساختن اسپیکر پرتابل میخوره,دوستان من این مدل برای ساخت اسپیکر پرتابل گرفت...,"[""کیفیت قابل قبول برد و آمپیلی فایر داخلی\r"",""...","[""نداشتن بلوتوث""]",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74993,329655,کفش راحتی زنانه لی نینگ مدل Bubble Face,FA,7093994,0,0,verified,\N,حتما بخرید,درسته قیمتش زیاده ولی کیفیتش عالیه _x000D_\nمن...,,,74992
74994,624592,فندک مدادی واته 8021005,HW,7094033,1,0,verified,\N,ارزش خرید داره,خودتون باید ابتکار داشته باشید کمی ارتقاش بدید...,"[""خوش دسته برای جوش های ظریف""]","[""نازل سوختش راحت خراب میشه""]",74995
74995,222488,اسپیکر مارلی مدل CHANT MINI,IT,7094148,0,1,verified,\N,عالی پیشنهاد میکنم,واقعا خیلی خوبه الان ربیق هارمن کاردن خیلی خوب...,"[""کیفیت بالا ""]","[""هیچی""]",74996
74996,48802,نرم افزار جامع جاوا اسکریپت,MA,7094364,0,0,verified,no_idea,خواب میره ادم به خدا,وقتی داره آموزش میده خواب میری به نظرم که خوب...,,"[""کند بودن تایپ\r"",""کند بودن اموزش\r"",""ضعیف بو...",74997


In [3]:
from pandas.core.frame import DataFrame
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import random
from scipy.sparse import csr_matrix

dff = DataFrame()
dff["user_id"] = df["user_id"]
dff["product_id"]= df["product_id"]
dff["content"] = df["comment"].str.cat(df["advantages"], sep = "-").str.cat(df["disadvantages"], sep = "-").str.cat(df["title"], sep = "-").str.cat(df["product_title"], sep = "-")


dff.head(100)

file_url = "https://raw.githubusercontent.com/shahabbadihi/IT/master/fa_stop_words.txt"
response = requests.get(file_url)

stop_words = []
with open("stopwords.txt", "wb") as filee:
  filee.write(response.content)

with open("stopwords.txt", "r") as filee:  
  for line in filee:
    stop_words.append(line)
stop_words = set(stop_words)

tf = TfidfVectorizer(analyzer='word',ngram_range=(1,3),min_df=0, stop_words=stop_words)
tfidf_matrix = tf.fit_transform(dff.loc[0:100000, 'content'].values.astype('U'))
print(tfidf_matrix)


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آباد', 'آخ', 'آخر', 'آخرها', 'آخه', 'آدمهاست', 'آرام', 'آره', 'آری', 'آزادانه', 'آسان', 'آسانی', 'آسیب', 'آشنایند', 'آشکارا', 'آقا', 'آقای', 'آقایان', 'آمد', 'آمدن', 'آمده', 'آمرانه', 'آن', 'آنان', 'آنانی', 'آنجا', 'آنرا', 'آنطور', 'آنقدر', 'آنكه', 'آنها', 'آنهاست', 'آنچنان', 'آنچه', 'آنکه', 'آنگاه', 'آهان', 'آهای', 'آور', 'آورد', 'آوردن', 'آورده', 'آوه', 'آيد', 'آی', 'آیا', 'آید', 'آیند', 'اتفاقا', 'اثر', 'اجراست', 'احتراما', 'احتمالا', 'احیانا', 'اخیر', 'اخیرا', 'اری', 'از', 'ازاین', 'ازجمله', 'ازش', 'اساسا', 'است', 'استفاد', 'استفاده', 'اسلامی', 'اش', 'اشتباها', 'اشکارا', 'اصلا', 'اصولا', 'اطلاعند', 'اعلام', 'اغلب', 'افزود', 'افسوس', 'اقل', 'اقلیت', 'اكنون', 'الا', 'الاسف', 'الان', 'البت', 'البته', 'الظاهر', 'الهی', 'الی', 'ام', 'اما', 'امروز', 'امروزه', 'امسال', 'امشب', 'امور', 'امیدوارم

  (0, 804220)	0.12258694874266902
  (0, 1130630)	0.1302236467453938
  (0, 1130662)	0.1201284811034217
  (0, 3720269)	0.12575646478446764
  (0, 1263975)	0.12575646478446764
  (0, 2619404)	0.1302236467453938
  (0, 2676753)	0.1302236467453938
  (0, 276676)	0.1302236467453938
  (0, 2172555)	0.12575646478446764
  (0, 1263752)	0.1302236467453938
  (0, 1796323)	0.1302236467453938
  (0, 1952358)	0.1302236467453938
  (0, 268643)	0.1302236467453938
  (0, 3339889)	0.12575646478446764
  (0, 2871265)	0.12258694874266902
  (0, 2850497)	0.1302236467453938
  (0, 2245948)	0.1302236467453938
  (0, 351748)	0.1302236467453938
  (0, 1483189)	0.1302236467453938
  (0, 1535974)	0.1302236467453938
  (0, 3080425)	0.1302236467453938
  (0, 1952568)	0.1302236467453938
  (0, 3608914)	0.1302236467453938
  (0, 2851206)	0.1302236467453938
  (0, 1356503)	0.1302236467453938
  :	:
  (99999, 2263457)	0.3271847327473835
  (99999, 1746451)	0.06038123021276558
  (99999, 2351192)	0.12019306741611066
  (99999, 732116)	0.083796

In [4]:
def recommendations(index):
  cos_sim = (cosine_similarity(tfidf_matrix[index],tfidf_matrix))
  sorted_sim = pd.Series(cos_sim[0]).sort_values(ascending = False)
  top_10 = list(sorted_sim.iloc[1:11].index)
  return top_10



[98749, 74997, 89223, 351, 45516, 15163, 26813, 99087, 89225, 17369]

In [7]:
print(recommendations(100))
print(recommendations(2222))
print(recommendations(3353))
print(recommendations(48452))
print(recommendations(52365))
print(recommendations(78965))
print(recommendations(120))
print(recommendations(440))
print(recommendations(5896))

[69771, 38323, 85513, 12047, 24119, 39935, 49315, 85959, 98338, 57622]
[15090, 47448, 43615, 29052, 82344, 12717, 65523, 41497, 8112, 75108]
[1762, 3354, 20148, 41527, 68842, 10071, 1759, 10774, 54001, 38319]
[13615, 59046, 19415, 57220, 24694, 9806, 62896, 54310, 66913, 66348]
[52250, 52259, 52325, 52255, 52297, 52225, 52314, 52345, 52305, 52374]
[80681, 41894, 83666, 82349, 66790, 71980, 84625, 90049, 75902, 93891]
[53610, 53572, 92611, 14379, 70642, 31965, 87250, 40913, 53564, 18194]
[42387, 18361, 10551, 12925, 46203, 50923, 18286, 52823, 17015, 40946]
[1802, 85875, 55435, 86862, 34924, 94179, 38720, 49632, 3762, 90703]


In [17]:
import random
for i in range(10):
  random_number=random.randint(0,100000)
  print('عنوان کالا:')
  print(df.loc[[random_number]].product_title)
  print('عناوین کالا های پیشنهادی : ')
  for recommend in recommendations(random_number):
    print(df.loc[[recommend]].product_title)

عنوان کالا:
63738    جا شمعی بنیکو مدل BE 17568
Name: product_title, dtype: object
عناوین کالا های پیشنهادی : 
68164    جا شمعی بنیکو مدل  STLH06-M
Name: product_title, dtype: object
95999    ظرف سرو بنیکو مدل BE 17560
Name: product_title, dtype: object
62308    جا کاردی بنیکو مدل BE 12344
Name: product_title, dtype: object
29837    ست قاشق و چنگال سالاد بنیکو مدل BE 17523
Name: product_title, dtype: object
64423    ظرف سرو بنیکو مدل BE 16521
Name: product_title, dtype: object
65277    پرده لیندر مدل Maputo
Name: product_title, dtype: object
50918    قفل کابلی مگا تولز مدل M66300
Name: product_title, dtype: object
24615    ظرف سرو بنیکو مدل BE-16644
Name: product_title, dtype: object
59108    ظرف سرو بنیکو مدل BE 16522
Name: product_title, dtype: object
98365    نگهدارنده صفحه کتاب مرکوری شاپ کد 050902
Name: product_title, dtype: object
عنوان کالا:
31196    شارژر همراه هوکو مدل B12C ظرفیت 13000 میلی آمپ...
Name: product_title, dtype: object
عناوین کالا های پیشنهادی : 
66943    شارژر هم